In [13]:
!pip install pathway panel --quiet

In [15]:
import pandas as pd #importing libraries
import pathway as pw
import panel as pn
import bokeh.plotting
from bokeh.models import HoverTool

pn.extension()

In [16]:
df =pd.read_csv('/content/dataset.csv') # read the data
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [17]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00


In [18]:
# Define the ordinal mapping
ordinal_mapping_traffic = {
    'low': 0,
    'average': 1,
    'high': 2
}
ordinal_mapping_vehicle ={
    'cycle':  0,
    'bike': 1,
    'car':  2,
    'truck': 3,

}

# Apply the mapping to convert the column
df['Traffic'] = df['TrafficConditionNearby'].map(ordinal_mapping_traffic)
df['VehicleTypeWeight'] = df['VehicleType'].map(ordinal_mapping_vehicle)
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,Traffic,VehicleTypeWeight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0,2
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0,1
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0,2
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0,2
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0,2
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0,2
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0,0
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0,2


In [19]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity","QueueLength","Traffic" ,"IsSpecialDay","VehicleTypeWeight" ]].to_csv("parking_stream.csv", index=False)

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int
    QueueLength: int
    Traffic: int
    IsSpecialDay: int
    VehicleTypeWeight: int

In [20]:
table=pw.io.csv.read(
  '/content/parking_stream.csv',
  schema=ParkingSchema,
  mode="streaming"
)

In [21]:
data= pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [22]:
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    hour_bin = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT%00:00:00")
)

In [29]:
import datetime

α = 1.0
β = 1.0
γ = 1.0
δ = 1.0
ε = 1.5
λ = 0.5

# Step 1: Window aggregation with Demand and join key
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        avg_occ=pw.reducers.avg(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        QueL=pw.reducers.avg(pw.this.QueueLength),
        Traffic=pw.reducers.avg(pw.this.Traffic),
        IsSpecialDay=pw.reducers.any(pw.this.IsSpecialDay),
        VehiWeight=pw.reducers.avg(pw.this.VehicleTypeWeight),
    )
    .with_columns(
        Demand=α * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap) +
               β * pw.this.QueL -
               γ * pw.this.Traffic +
               δ * pw.this.IsSpecialDay +
               ε * pw.this.VehiWeight,
        join_key="global"
    )
)

# Step 2: Compute global min and max
demand_stats = (
    delta_window
    .reduce(
        join_key=pw.reducers.any(pw.this.join_key),
        min_demand=pw.reducers.min(pw.this.Demand),
        max_demand=pw.reducers.max(pw.this.Demand),
    )
)

# Step 3: Join with explicit condition using pw.left / pw.right
delta_window_final = (
    delta_window
    .join(demand_stats, pw.left.join_key == pw.right.join_key)
    .select(
        t=pw.left.t,
        SystemCodeNumber=pw.left.SystemCodeNumber,
        occ_max=pw.left.occ_max,
        occ_min=pw.left.occ_min,
        avg_occ=pw.left.avg_occ,
        cap=pw.left.cap,
        QueL=pw.left.QueL,
        Traffic=pw.left.Traffic,
        IsSpecialDay=pw.left.IsSpecialDay,
        VehiWeight=pw.left.VehiWeight,
        Demand=pw.left.Demand,
        NormalizedDemand=(pw.left.Demand - pw.right.min_demand) / (pw.right.max_demand - pw.right.min_demand + 1e-6),
        Price=10 * (1 + λ * ((pw.left.Demand - pw.right.min_demand) / (pw.right.max_demand - pw.right.min_demand + 1e-6)))
    )
)


# Step 4: Write to CSV
pw.io.csv.write(delta_window_final, "output_stream.csv")


In [30]:
# 🎨 Plotting function
def price_plotter(source, code):
    fig = bokeh.plotting.figure(
        height=300, width=800, x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset",
        title=f"Price Trend – {code}"
    )
    fig.line("t", "Price", source=source, line_width=2, color="navy")
    fig.circle("t", "Price", source=source, size=5, color="red")
    fig.add_tools(HoverTool(
        tooltips=[("Time", "@t{%F %H:%M}"), ("Price", "@Price{0.00}")],
        formatters={"@t": "datetime"}
    ))
    return fig

# 📊 Generate per-lot plots
system_code_list = sorted(df['SystemCodeNumber'].unique())

plots = [
    delta_window_final
        .filter(pw.this.SystemCodeNumber == code)
        .select(pw.this.t, pw.this.Price)
        .plot(lambda source, code=code: price_plotter(source, code), sorting_col="t")
    for code in system_code_list
]

dashboard = pn.Column(*[
    pn.Column(f"### {code}", plot)
    for code, plot in zip(system_code_list, plots)
])

dashboard

Column
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [13] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)

In [ ]:
pw.run()

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished          5409        18368    output            0       None                          
    PythonRead…     finished          5414        18368                                                            
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    